<a href="https://colab.research.google.com/github/kyokicchi/ggl_an_codes/blob/master/NNmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train model

In [0]:
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from keras import models
from keras import layers

from keras.models import load_model



In [0]:
drive.mount('/content/gdrive')

In [0]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

In [0]:
%%time
filename = 'model_NN.h5'
model = load_model(filename)


In [0]:

df_y = df_user_train['transactionRevenue']
df_x = df_user_train.drop(['transactionRevenue'], axis=1)
df_tgt = df_user_test.drop(['transactionRevenue'], axis=1)

df_x.shape, df_y.shape, df_tgt.shape


In [0]:
%%time

from sklearn.metrics import mean_squared_error

pred = model.predict(df_x)
mse = mean_squared_error(df_y, pred)
rmse = np.sqrt(mse)
print(rmse)

In [0]:
from keras import optimizers


def build_model(N_col):
    model = models.Sequential()
    model.add(layers.Dense(120, activation='relu', input_shape=(N_col,)))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(30, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dropout(0.1))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer=optimizers.adam(lr=0.01), loss='mse', metrics=['mae'])
    return model

In [0]:
%%time
epochs = 200
batch_size = df_x.shape[0] // 5
validation_split = 0.1

model = build_model(df_x.shape[1])

history = model.fit(df_x,
                    df_y,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_split = validation_split,
                    verbose=1)

In [0]:
#history.history

In [0]:
%%time
pred_tgt = model.predict(df_tgt)


In [0]:
pred_tgt.shape, pred_tgt.max(), pred_tgt.min(), len(pred_tgt[pred_tgt>=1.5])/len(pred_tgt)

In [0]:
print(len(df_y[df_y<=0.1]) - len(df_y[df_y==0]))

print(len(pred_tgt[pred_tgt<=0.1]) - len(pred_tgt[pred_tgt==0]))

print(len(df_y[df_y>0]) / len(df_y) )

print(len(pred_tgt[pred_tgt>1]) / len(pred_tgt) )

In [0]:
pred_tgt[pred_tgt<0] = 0
df_out = pd.DataFrame(index = df_tgt.index)
df_out['PredictedLogRevenue'] = pred_tgt


In [0]:
df_out.head()

In [0]:
df_out.describe()

In [0]:
%%time
df_out.to_csv('output_NN.csv')

In [0]:
%%time
model.save(filename)

#files.download(filename)